In [1]:
import pickle
import numpy as np
import pandas as pd

In [2]:
Asset_PATH = "/home/zhimin90/DePaul/CSC695/Github/CSC695-Masters-Research-on-Neural-Network-Time-Series-Attention-Models/Asset/"

#### Load Gaussian KDE timeseries

In [3]:
density_matrix_t_series = pickle.load(open(Asset_PATH  + 'density_matrix_t_series.pickle', 'rb'))

In [4]:
density_matrix_t_series.shape

(198, 1000, 1000)

In [5]:
print(np.max(density_matrix_t_series))
print(np.min(density_matrix_t_series))

1373.5868386259936
-1.9779578676050593e-13


In [6]:
density_matrix_t_series.shape[0]

198

In [7]:
train, valid = (60,20)
test = 20

total_len = density_matrix_t_series.shape[0]
train_index = range(0,int(train/100*total_len))

In [8]:
valid_index = range(train_index[-1]+1,train_index[-1] + int(valid/100*total_len))
test_index = range(valid_index[-1]+1,valid_index[-1] + int(test/100*total_len))

In [9]:
flattened_matrix_np = np.reshape(density_matrix_t_series, 
                                 (density_matrix_t_series.shape[0],
                                  density_matrix_t_series.shape[1]*density_matrix_t_series.shape[2]))
flattened_matrix_np.shape

(198, 1000000)

In [31]:
from sklearn.preprocessing import MinMaxScaler #,StandardScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(flattened_matrix_np[train_index])


In [32]:
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
X_train.shape

(118, 1, 1000000)

In [33]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],int(X_train.shape[2]**(1/2)),int(X_train.shape[2]**(1/2)),1)
X_train.shape

(118, 1, 1000, 1000, 1)

In [13]:
print(np.max(X_train))
print(np.min(X_train))

1.0000000000000002
0.0


#### We transform the whole train, valid, test based on scaler fitted on train

In [35]:
scaled_flattened_matrix = scaler.transform(flattened_matrix_np)

#### We reshape to (sample, timestep, inputshape)

In [36]:
scaled_flattened_matrix = scaled_flattened_matrix.reshape(scaled_flattened_matrix.shape[0],
                                                           1,
                                                           scaled_flattened_matrix.shape[1])
scaled_flattened_matrix.shape

(198, 1, 1000000)

#### We set the target to be 7 days aggregated KDE or 1 single frame forward

In [37]:
def reshape(X_train):
    if len(X_train.shape)>2:
        return X_train.reshape(X_train.shape[0],X_train.shape[1],int(X_train.shape[2]**(1/2)),int(X_train.shape[2]**(1/2)),1)
    else:
        return X_train.reshape(X_train.shape[0],int(X_train.shape[1]**(1/2)),int(X_train.shape[1]**(1/2)),1)

In [38]:
Y_train = scaled_flattened_matrix[np.array(train_index)+1].reshape(
    scaled_flattened_matrix[np.array(train_index)+1].shape[0],scaled_flattened_matrix[np.array(train_index)+1].shape[2])
Y_train = reshape(Y_train)

X_valid = scaled_flattened_matrix[np.array(valid_index)]
X_valid = reshape(X_valid)

Y_valid = scaled_flattened_matrix[np.array(valid_index)+1].reshape(
    scaled_flattened_matrix[np.array(valid_index)+1].shape[0],scaled_flattened_matrix[np.array(valid_index)+1].shape[2])
Y_valid = reshape(Y_valid)

X_test = scaled_flattened_matrix[np.array(test_index)]
X_test = reshape(X_test)

Y_test = scaled_flattened_matrix[np.array(test_index)+1].reshape(
    scaled_flattened_matrix[np.array(test_index)+1].shape[0],scaled_flattened_matrix[np.array(test_index)+1].shape[2])
Y_test = reshape(Y_test)

In [39]:
print(X_train[0:2])
print('-'*100)
print(Y_train[0])

[[[[[0.60441593]
    [0.58182973]
    [0.55164859]
    ...
    [0.38867659]
    [0.43911245]
    [0.47768694]]

   [[0.54314872]
    [0.51742145]
    [0.46258717]
    ...
    [0.22645183]
    [0.20301129]
    [0.17867648]]

   [[0.56163515]
    [0.55776216]
    [0.55777539]
    ...
    [0.34234551]
    [0.40902312]
    [0.42605115]]

   ...

   [[0.3551999 ]
    [0.2996704 ]
    [0.23973855]
    ...
    [0.24064679]
    [0.29429379]
    [0.3342973 ]]

   [[0.30525529]
    [0.35188671]
    [0.39969033]
    ...
    [0.28016812]
    [0.26736888]
    [0.25722946]]

   [[0.04280326]
    [0.08066387]
    [0.08015056]
    ...
    [0.32541977]
    [0.31968994]
    [0.26762363]]]]



 [[[[0.7817593 ]
    [0.78284985]
    [0.78405589]
    ...
    [0.55029251]
    [0.54730131]
    [0.55789942]]

   [[0.88465522]
    [0.88893822]
    [0.8769544 ]
    ...
    [0.34876011]
    [0.3343945 ]
    [0.34488595]]

   [[0.46993552]
    [0.47346894]
    [0.46545939]
    ...
    [0.55019749]
    [0.55770246]

In [40]:
print(X_valid[0:2])
print('-'*100)
print(Y_valid[0])

[[[[[ 0.63873992]
    [ 0.67797451]
    [ 0.71745821]
    ...
    [ 0.66047035]
    [ 0.72086383]
    [ 0.77389617]]

   [[ 0.53008569]
    [ 0.53357382]
    [ 0.53578384]
    ...
    [ 0.17650283]
    [ 0.22193099]
    [ 0.30124343]]

   [[ 0.20233666]
    [ 0.22734124]
    [ 0.28821334]
    ...
    [ 0.27555832]
    [ 0.33994046]
    [ 0.3723761 ]]

   ...

   [[ 0.10712916]
    [ 0.04049415]
    [-0.01919522]
    ...
    [ 0.28277002]
    [ 0.35836427]
    [ 0.40042785]]

   [[ 0.34337304]
    [ 0.29948787]
    [ 0.25655559]
    ...
    [ 0.40692234]
    [ 0.4468157 ]
    [ 0.4905469 ]]

   [[ 0.26399218]
    [ 0.2122477 ]
    [ 0.13850271]
    ...
    [ 0.44575125]
    [ 0.49944353]
    [ 0.5345717 ]]]]



 [[[[ 0.48523906]
    [ 0.51866757]
    [ 0.56361375]
    ...
    [-0.06757383]
    [ 0.01401152]
    [ 0.14172242]]

   [[ 0.56052966]
    [ 0.54366578]
    [ 0.52869966]
    ...
    [ 0.47495116]
    [ 0.43401174]
    [ 0.35944391]]

   [[ 0.58735254]
    [ 0.58747072]
    [ 0.

In [41]:
print(X_train.shape)
print(Y_train.shape)
print(X_valid.shape)
print(Y_valid.shape)
print(X_test.shape)
print(Y_test.shape)

(118, 1, 1000, 1000, 1)
(118, 1000, 1000, 1)
(38, 1, 1000, 1000, 1)
(38, 1000, 1000, 1)
(38, 1, 1000, 1000, 1)
(38, 1000, 1000, 1)


In [42]:
import os, datetime
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, GaussianNoise, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Concatenate, Input, LSTM, RepeatVector, TimeDistributed
import tensorflow.keras.layers as layers

In [43]:
checkpoint_filepath = Asset_PATH+'checkpoints/'

## Top half of AutoEncoder LSTM Model with > shaped LSTM layers

In [48]:
serie_size, timestep, n_features_row, n_features_column, no_channel = X_train.shape

epochs = 100
batch = 48
lr = 0.1
loss='mae'
METRIC_ACCURACY = 'mae'

cnn = Sequential()
cnn.add(Conv2D(1, (2,2), activation='relu', padding='same', input_shape=(n_features_row, n_features_column,no_channel )))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Flatten())



lstm_model = Sequential()
lstm_model.add(TimeDistributed(cnn))
lstm_model.add(LSTM(20))
#lstm_model.add(LSTM(60, activation='relu', return_sequences=True))
#lstm_model.add(LSTM(10, activation='relu'))
lstm_model.add(Dense(10, kernel_initializer='glorot_normal', activation='relu'))
#lstm_model.add(Dense(10, kernel_initializer='glorot_normal', activation='relu'))
lstm_model.add(Dense(Y_train.shape[1]))
#lstm_model.summary()

adam = optimizers.Adam(lr)
lstm_model.compile(loss=loss,
                   metrics=[METRIC_ACCURACY], 
                   optimizer=adam)
#lstm_model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [46]:
EPOCHS = 5

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
    min_delta=0,
    patience=EPOCHS*10,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=True,)

logdir = os.path.join("./logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_cbk = tf.keras.callbacks.TensorBoard(log_dir=logdir)

modelfilename = checkpoint_filepath + '_'+\
                    datetime.datetime.now().strftime("%Y%m%d-%H%M%S")\
                    + "_model.hdf5"

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    modelfilename,
    save_weights_only=False,
    monitor='val_mae',#'val_accuracy',
    mode='min',
    verbose=1,
    period = 1,
    save_best_only=True)



lstm_model_history = lstm_model.fit(X_train,Y_train, 
                             validation_data=(X_valid, Y_valid),
                             epochs=20, 
                             #batch_size=batch, 
                             #callbacks=[tensorboard_cbk,
                             #hp.KerasCallback(logdir, hparams),
                             #earlystop_callback,
                             #model_checkpoint_callback]
                                   )

Train on 118 samples, validate on 38 samples
Epoch 1/20
 32/118 [=======>......................] - ETA: 9s

InvalidArgumentError:  Incompatible shapes: [32,1000] vs. [32,1000,1000,1]
	 [[node loss/output_1_loss/sub (defined at <ipython-input-46-0adb98e35bba>:31) ]] [Op:__inference_distributed_function_3245]

Function call stack:
distributed_function


In [ ]:
valid_loss, valid_acc = lstm_model.evaluate(X_valid, Y_valid, verbose=2) # 5/2020 nt: use validation set
print ("valid_accuracy_MAE=%s, valid_loss=%s" % (valid_acc, valid_loss))

In [ ]:
Predict = lstm_model.predict(X_valid)

In [ ]:
X_valid[0]

In [ ]:
X_valid[1]

In [ ]:
Predict.shape

In [ ]:
Predict[0]

In [ ]:
Predict[1]